# LLM Zoomcamp Homework 01

## Q1. Running Elastic

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

```curl localhost:9200```

What's the ```version.build_hash``` value?

In [1]:
!curl localhost:9200

{
  "name" : "94082acc643e",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "rmSpIbD7QoKvuSxJ3Q_aDQ",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## Getting the data

In [2]:
# !pip install requests

import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the ```course``` field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

```pip install elasticsearch```

Which function do you use for adding your data to elastic?

In [3]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

es_client = Elasticsearch('http://localhost:9200', verify_certs=False)

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

if not es_client.indices.exists(index=index_name):
    es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only ```question``` and ```text``` fields and give ```question``` a boost of 4, and use ```"type": "best_fields"```.

What's the score for the top ranking result?

In [4]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do execute a command on a Kubernetes pod?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

# response['hits']['max_score']
response['hits']['hits'][0]['_score']

44.58162

## Q4. Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from ```machine-learning-zoomcamp```.

Return 3 results. What's the 3rd question returned by the search engine?

In [5]:
question = 'How do copy a file to a Docker container?'

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][2]['_source']['question']

'How do I debug a docker container?'

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (```\n\n```)

```
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do copy a file to a Docker container?" question to construct a prompt using the template below

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)


In [6]:
entries = []
for hit in response['hits']['hits']:
    entry = f"""Q: {hit['_source']['question']}\nA:{hit['_source']['text']}""".strip()
    entries.append(entry)

context = '\n\n'.join(entries)

prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

len(prompt)

1303

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses ```tiktoken``` for tokenization:

```pip install tiktoken```

Let's calculate the number of tokens in our query:

```encoding = tiktoken.encoding_for_model("gpt-4o")```

Use the ```encode``` function. How many tokens does our prompt have?

In [7]:
# !pip install tiktoken

import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

len(encoding.encode(prompt))

296

In [8]:
for token in encoding.encode(prompt):
    print(token, repr(encoding.decode([token])))

63842 "You're"
261 ' a'
4165 ' course'
14029 ' teaching'
29186 ' assistant'
13 '.'
30985 ' Answer'
290 ' the'
150339 ' QUESTION'
4122 ' based'
402 ' on'
290 ' the'
31810 ' CONT'
8099 'EXT'
591 ' from'
290 ' the'
40251 ' FAQ'
7862 ' database'
558 '.\n'
8470 'Use'
1606 ' only'
290 ' the'
19719 ' facts'
591 ' from'
290 ' the'
31810 ' CONT'
8099 'EXT'
1261 ' when'
55959 ' answering'
290 ' the'
150339 ' QUESTION'
364 '.\n\n'
107036 'QUESTION'
25 ':'
3253 ' How'
621 ' do'
5150 ' copy'
261 ' a'
1974 ' file'
316 ' to'
261 ' a'
91238 ' Docker'
9282 ' container'
1715 '?\n\n'
10637 'CON'
50738 'TEXT'
734 ':\n'
48 'Q'
25 ':'
3253 ' How'
621 ' do'
357 ' I'
15199 ' debug'
261 ' a'
62275 ' docker'
9282 ' container'
3901 '?\n'
32 'A'
25 ':'
35423 'Launch'
290 ' the'
9282 ' container'
3621 ' image'
306 ' in'
25383 ' interactive'
6766 ' mode'
326 ' and'
151187 ' overriding'
290 ' the'
7251 ' entry'
4859 'point'
11 ','
813 ' so'
484 ' that'
480 ' it'
13217 ' starts'
261 ' a'
38615 ' bash'
6348 ' command'